# The Price is Right

## Day 8 Order of Play

Part 1: Modal.com and SpecialistAgent  
Part 2: RAG, FrontierAgent, Ensemble Agent  
Part 3: ScannerAgent, MessengerAgent  
Part 4: AutonomousPlannerAgent and DealAgentFramework  
Part 5: The Price Is Right Finale


Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [16]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection
import logging
import requests
load_dotenv(override=True)

# 🔁 CHANGED: OpenRouter client (OpenAI compatible)
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

# 🔁 CHANGED: OpenRouter model
MODEL = "meta-llama/llama-3.1-8b-instruct"


In [5]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 3/3 [00:43<00:00, 14.36s/it]


In [6]:
len(deals)

30

In [7]:
deals[10].describe()

'Title: Refurb Apple 10.2" 128GB iPad (7th Gen; 2019) for $160\nDetails: That\'s the best price we could find for a refurb by $42. It comes with a 90-day warranty. Buy Now at UntilGone\nFeatures: 2160x1620 10.2" Retina display 8MP camera updates to iOS 16 Model: MW772LL/A\nURL: https://www.dealnews.com/products/Apple/Apple-10-2-128-GB-iPad-7-th-Gen-2019/497292.html?iref=rss-c39'

### We are going to ask GPT-5-mini to summarize deals and identify their price

In [8]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format:

[
  {
    "title": "...",
    "product_description": "...",
    "price": 123,
    "url": "https://..."
  }
]

Each deal MUST contain title, product_description, price, and url.
You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""


USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Each deal must include: title, product_description, price, and url.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [9]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [10]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Each deal must include: title, product_description, price, and url.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Shokz OpenFit 2 Plus Earbuds for $150 + free shipping
Details: Most other stores charge $200. Buy Now at Best Buy
Features: 
URL: https://www.dealnews.com/Shokz-Open-Fit-2-Plus-Earbuds-for-150-free-shipping/21803263.html?iref=rss-c142

Title: TCL 65QM5K QM5K Series 65" 4K UHD HDR LED Google TV (2025) for $450 +

In [11]:
import json

response = openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    temperature=0
)

content = response.choices[0].message.content
data = json.loads(content)

results = DealSelection(deals=data)


In [12]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


3840x2160 (4K) resolution 60Hz refresh rate HDR 10+ three HDMI ports Model: 65QM5K
450.0
https://www.dealnews.com/products/TCL/TCL-65-QM5-K-QM5-K-Series-65-4-K-UHD-HDR-LED-Google-TV-2025/495416.html?iref=rss-c142

Intel i7-4770HQ Crystal Well 2.2GHz quad-core CPU 15.4" 2880x1800 Retina display 16GB RAM & 256GB PCIe flash storage Mac OS Model: MJLQ2LL/A
235.0
https://www.dealnews.com/products/Apple/Refurb-Apple-Mac-Book-Pro-i7-15-4-Laptop-2015/835.html?iref=rss-c39

functions as a laptop or a tablet Intel Core Ultra 5 Processor 226V 2.1GHz Lunar Lake 8-core CPU 16" 1920x1200 touchscreen 16GB RAM & 512GB SSD Windows 11
600.0
https://www.dealnews.com/Lenovo-Yoga-7-i-Ultra-5-16-Touch-2-in-1-Laptop-w-512-GB-SSD-for-600-free-shipping/21802964.html?iref=rss-c39

Intel Core i3-N305 processor 14" touchscreen IPS widescreen LED FHD display 1920x1080p resolution Intel WiFi 6E and Bluetooth 1080p webcam with privacy shutter Model: NX.JKBAA.001
178.0
https://www.dealnews.com/products/Acer/Acer-Chro

In [17]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [18]:
from agents.scanner_agent import ScannerAgent

In [19]:
agent = ScannerAgent()
result = agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 30 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling Groq
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from Groq


In [22]:
result

DealSelection(deals=[Deal(product_description='Shokz OpenFit 2 Plus Earbuds feature IP67 water and sweat resistance, up to 8 hours of battery life, and wireless charging. They provide great sound quality with deep bass and a clear Treble response, and their ergonomic design fits comfortably and secure', price=150.0, url='https://www.dealnews.com/Shokz-Open-Fit-2-Plus-Earbuds-for-150-free-shipping/21803263.html?iref=rss-c142'), Deal(product_description='The TCL 65QM5K QM5K Series 65" 4K UHD HDR LED Google TV (2025) features a 3840x2160 resolution, 60Hz refresh rate, and HDR 10+ support. It has three HDMI ports and is a part of TCL\'s QM5K Series', price=450.0, url='https://www.dealnews.com/products/TCL/TCL-65-QM5-K-QM5-K-Series-65-4-K-UHD-HDR-LED-Google-TV-2025/495416.html?iref=rss-c142'), Deal(product_description='The MoveSpeed Z70 100W 70,000mAh Portable Power Bank features three-level lighting and SOS mode, as well as dual USB-C and dual USB-A output. It provides 100 watts of quick p